In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 


# sklearn preprocessing for dealing with categorical variables
import sklearn
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, mean_squared_error, accuracy_score,roc_auc_score,recall_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import KNNImputer
from sklearn.metrics import make_scorer

from tqdm import tqdm

import h2o
from h2o.estimators import H2ORandomForestEstimator


from xgboost import XGBClassifier

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')



# matplotlib and seaborn for plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import mlflow
import mlflow.sklearn




import gc
import time
import warnings

from contextlib import contextmanager
from lightgbm import LGBMClassifier

## Data Train

In [2]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    
    # Nombre de colonnes supprimées (colonnes catégorielles remplacées par les dummies)
    nb_columns_removed = len(categorical_columns)

    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns,nb_columns_removed

In [3]:
# Preprocess application_train.csv and application_test.csv
def application_train(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\application_test.csv', nrows= num_rows)
    ##test_df = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\application_test.csv', nrows= num_rows)
    print("Train samples: {}".format(len(df)))
    print("taille du df initial: {}".format(df.shape)) 
    #df = pd.concat([df, test_df], ignore_index=True)
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    binary_cols = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
    for bin_feature in binary_cols:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols,nb_suppr_cols = one_hot_encoder(df, nan_as_category=True)
    print("Nouvelles colonnes créés encoder: {}".format(len(cat_cols)))
    print("Anciennes colonnes supprimées: {}".format(nb_suppr_cols))

    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

    print("Nouvelles colonnes créés ratio: {}".format(5))

    # Identify non-categorical columns
    encoded_columns = binary_cols + cat_cols
    # Use set operations to exclude 'SK_ID_CURR' and 'TARGET'
    non_categorical_cols = df.columns.difference(set(encoded_columns).union({'SK_ID_CURR', 'TARGET'}))

    


        
    gc.collect()

    # sauvegarder les colonnes encodées
    encoded_columns = binary_cols + cat_cols
    
    return df, encoded_columns
    

In [4]:
df_data,col_encod=application_train(num_rows = None, nan_as_category = False)

Train samples: 48744
taille du df initial: (48744, 121)
Nouvelles colonnes créés encoder: 144
Anciennes colonnes supprimées: 13
Nouvelles colonnes créés ratio: 5


In [5]:
df_data.shape

(48744, 257)

In [6]:
df_data.head

<bound method NDFrame.head of        SK_ID_CURR  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  CNT_CHILDREN  \
0          100001            0             0                0             0   
1          100005            1             0                0             0   
2          100013            1             1                0             0   
3          100028            0             0                0             2   
4          100038            1             1                1             1   
...           ...          ...           ...              ...           ...   
48739      456221            0             0                0             0   
48740      456222            0             0                1             2   
48741      456223            0             1                0             1   
48742      456224            1             0                1             0   
48743      456250            0             1                1             0   

       AMT_INCOME_TOT

## Data Bureau and Balance

In [7]:
import pandas as pd
import numpy as np
import gc
from sklearn.preprocessing import StandardScaler

# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows=None, nan_as_category=True):
    bureau = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\bureau.csv', nrows=num_rows)
    bb = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\bureau_balance.csv', nrows=num_rows)

    print("taille du df bureau initial: {}".format(bureau.shape))
    print("taille du df bb: {}".format(bb.shape))      
    
    # One-hot encoding for categorical variables
    bb, bb_cat,bb_suppr_cols = one_hot_encoder(bb, nan_as_category=True)
    bureau, bureau_cat,bureau_suppr_cols = one_hot_encoder(bureau, nan_as_category=True)
    print("Nouvelles colonnes bb créés encoder: {}".format(len(bb_cat)))
    print("Anciennes colonnes bb supprimées: {}".format(bb_suppr_cols))
    print("Nouvelles colonnes bureau créés encoder: {}".format(len(bureau_cat)))
    print("Anciennes colonnes bb supprimées: {}".format(bureau_suppr_cols))
    
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    
    # Merge aggregated bureau balance data
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
    
    # Garbage collection
    del bb, bb_agg
    gc.collect()

    # Bureau and bureau_balance numeric features aggregations
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat:
        cat_aggregations[cat] = ['mean']
    for cat in bb_cat:
        cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

    # Identify non-categorical columns
    encoded_columns = bb_cat + bureau_cat
    # Exclure explicitement 'SK_ID_CURR' des colonnes à scaler
    non_categorical_cols = bureau_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])

     
    
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')

    # Garbage collection
    del active, active_agg
    gc.collect()
    
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')

    # Après la jointure, réinitialiser l'index de bureau_agg pour récupérer 'SK_ID_CURR' comme colonne
    bureau_agg = bureau_agg.reset_index()

    # Cleanup
    del closed, closed_agg, bureau
    gc.collect()

    print("taille du df bureau_agg: {}".format(bureau_agg.shape))  
    
    return bureau_agg


In [8]:
df_bureau=bureau_and_balance(num_rows = None, nan_as_category = True)

taille du df bureau initial: (1716428, 17)
taille du df bb: (27299925, 3)
Nouvelles colonnes bb créés encoder: 9
Anciennes colonnes bb supprimées: 1
Nouvelles colonnes bureau créés encoder: 26
Anciennes colonnes bb supprimées: 3
taille du df bureau_agg: (305811, 117)


In [9]:
# Mettre 'SK_ID_CURR' en index de df_bureau
df_bureau.set_index('SK_ID_CURR', inplace=True)
# Faire la jointure
df_data = df_data.join(df_bureau, how='left', on='SK_ID_CURR')

In [10]:
df_data.shape

(48744, 373)

In [11]:
df_data.head

<bound method NDFrame.head of        SK_ID_CURR  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  CNT_CHILDREN  \
0          100001            0             0                0             0   
1          100005            1             0                0             0   
2          100013            1             1                0             0   
3          100028            0             0                0             2   
4          100038            1             1                1             1   
...           ...          ...           ...              ...           ...   
48739      456221            0             0                0             0   
48740      456222            0             0                1             2   
48741      456223            0             1                0             1   
48742      456224            1             0                1             0   
48743      456250            0             1                1             0   

       AMT_INCOME_TOT

## Data Previous

In [12]:
# Preprocess previous_applications.csv
def previous_applications(num_rows=None, nan_as_category=True):
    prev = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\previous_application.csv', nrows=num_rows)
    prev, cat_cols,prev_suppr_cols = one_hot_encoder(prev, nan_as_category=nan_as_category)

    # Remplacer les valeurs 365.243 par NaN
    days_columns = [
        'DAYS_FIRST_DRAWING', 
        'DAYS_FIRST_DUE', 
        'DAYS_LAST_DUE_1ST_VERSION', 
        'DAYS_LAST_DUE', 
        'DAYS_TERMINATION'
    ]
    for col in days_columns:
        prev[col].replace(365243, np.nan, inplace=True)

    # Ajouter une nouvelle fonctionnalité : pourcentage de demande / valeur reçue
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']

    # Agrégations pour les caractéristiques numériques
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }

    # Agrégations pour les caractéristiques catégorielles
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']

    # Agrégation des données
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

    # Traitement des valeurs infinies
    prev_agg.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Imputation des valeurs manquantes
    prev_agg.fillna(prev_agg.mean(), inplace=True)

    # Identifier les colonnes non catégorielles à mettre à l'échelle
    encoded_columns = pd.Index(cat_cols)  # Convertir en Index
    non_categorical_cols = prev_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])  # Exclure SK_ID_CURR
    


    # Applications précédentes : applications approuvées - uniquement les caractéristiques numériques
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')

    # Applications précédentes : applications refusées - uniquement les caractéristiques numériques
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')

    # Nettoyage de la mémoire
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()

    return prev_agg


In [13]:
df_previous=previous_applications(num_rows = None, nan_as_category = True)

In [14]:
df_previous.info()

<class 'pandas.core.frame.DataFrame'>
Index: 338857 entries, 100001 to 456255
Columns: 249 entries, PREV_AMT_ANNUITY_MIN to REFUSED_CNT_PAYMENT_SUM
dtypes: float64(245), int64(4)
memory usage: 646.3 MB


In [15]:
df_data = df_data.join(df_previous, how='left', on='SK_ID_CURR')

In [16]:
df_data.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,REFUSED_HOUR_APPR_PROCESS_START_MAX,REFUSED_HOUR_APPR_PROCESS_START_MEAN,REFUSED_RATE_DOWN_PAYMENT_MIN,REFUSED_RATE_DOWN_PAYMENT_MAX,REFUSED_RATE_DOWN_PAYMENT_MEAN,REFUSED_DAYS_DECISION_MIN,REFUSED_DAYS_DECISION_MAX,REFUSED_DAYS_DECISION_MEAN,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM
0,100001,0,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100005,1,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100013,1,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100028,0,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100038,1,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 622 entries, SK_ID_CURR to REFUSED_CNT_PAYMENT_SUM
dtypes: bool(144), float64(436), int64(42)
memory usage: 184.5 MB


## Pos_Cash_Balance

In [18]:
# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows=None, nan_as_category=True):
    pos = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\POS_CASH_balance.csv', nrows=num_rows)
    pos, cat_cols,pos_suppr_cols = one_hot_encoder(pos, nan_as_category=nan_as_category)

    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Compter les comptes de POS Cash
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    
    # Traitement des valeurs infinies
    pos_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Imputation des valeurs manquantes
    pos_agg.fillna(pos_agg.mean(), inplace=True)

   # Identify columns that should not be scaled (categorical + SK_ID_CURR)
    encoded_columns = cat_cols  # Assuming cat_cols contains all categorical columns
    non_categorical_cols = pos_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])  # Exclure SK_ID_CURR

    
   
    # Nettoyage de la mémoire
    del pos
    gc.collect()

    return pos_agg

In [19]:
df_Pos_Cash=pos_cash(num_rows = None, nan_as_category = True)

In [20]:
df_Pos_Cash.info()

<class 'pandas.core.frame.DataFrame'>
Index: 337252 entries, 100001 to 456255
Data columns (total 18 columns):
 #   Column                                               Non-Null Count   Dtype  
---  ------                                               --------------   -----  
 0   POS_MONTHS_BALANCE_MAX                               337252 non-null  int64  
 1   POS_MONTHS_BALANCE_MEAN                              337252 non-null  float64
 2   POS_MONTHS_BALANCE_SIZE                              337252 non-null  int64  
 3   POS_SK_DPD_MAX                                       337252 non-null  int64  
 4   POS_SK_DPD_MEAN                                      337252 non-null  float64
 5   POS_SK_DPD_DEF_MAX                                   337252 non-null  int64  
 6   POS_SK_DPD_DEF_MEAN                                  337252 non-null  float64
 7   POS_NAME_CONTRACT_STATUS_Active_MEAN                 337252 non-null  float64
 8   POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN         3

In [21]:
df_Pos_Cash.head()

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9
100002,-1,-10.000000,19,0,0.000000,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19
100003,-18,-43.785714,28,0,0.000000,0,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28
100004,-24,-25.500000,4,0,0.000000,0,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4
100005,-15,-20.000000,11,0,0.000000,0,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11


In [22]:
df_data = df_data.join(df_Pos_Cash, how='left', on='SK_ID_CURR')

In [23]:
df_data.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT
0,100001,0,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0
1,100005,1,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0
2,100013,1,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,0.0,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0
3,100028,0,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,0.0,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0
4,100038,1,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,0.0,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0


In [24]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 640 entries, SK_ID_CURR to POS_COUNT
dtypes: bool(144), float64(454), int64(42)
memory usage: 191.2 MB


## installments_payments

In [25]:
# Preprocess installments_payments.csv
def installments_payments(num_rows=None, nan_as_category=True):
    ins = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\installments_payments.csv', nrows=num_rows)
    ins, cat_cols,ins_suppr_ = one_hot_encoder(ins, nan_as_category=nan_as_category)

    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)

    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    
    for cat in cat_cols:
        aggregations[cat] = ['mean']
        
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Compter les comptes d'installations
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    
    # Traitement des valeurs infinies
    ins_agg.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Imputation des valeurs manquantes
    ins_agg.fillna(ins_agg.mean(), inplace=True)

    # Identify columns that should not be scaled (categorical + SK_ID_CURR)
    encoded_columns = cat_cols  # Assuming cat_cols contains all categorical columns
    non_categorical_cols = ins_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])  # Exclure SK_ID_CURR

    

    # Nettoyage de la mémoire
    del ins
    gc.collect()

    return ins_agg


In [26]:
df_installments=installments_payments(num_rows = None, nan_as_category = True)

In [27]:
df_installments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339587 entries, 100001 to 456255
Data columns (total 26 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE  339587 non-null  int64  
 1   INSTAL_DPD_MAX                         339587 non-null  float64
 2   INSTAL_DPD_MEAN                        339587 non-null  float64
 3   INSTAL_DPD_SUM                         339587 non-null  float64
 4   INSTAL_DBD_MAX                         339587 non-null  float64
 5   INSTAL_DBD_MEAN                        339587 non-null  float64
 6   INSTAL_DBD_SUM                         339587 non-null  float64
 7   INSTAL_PAYMENT_PERC_MAX                339587 non-null  float64
 8   INSTAL_PAYMENT_PERC_MEAN               339587 non-null  float64
 9   INSTAL_PAYMENT_PERC_SUM                339587 non-null  float64
 10  INSTAL_PAYMENT_PERC_VAR                339587 non-null  

In [28]:
df_data = df_data.join(df_installments, how='left', on='SK_ID_CURR')

In [29]:
df_data.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
0,100001,0,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,5885.132143,41195.925,3951.000,17397.900,5885.132143,41195.925,-1628.0,-2195.000000,-15365.0,7.0
1,100005,1,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,6240.205000,56161.845,4813.200,17656.245,6240.205000,56161.845,-470.0,-609.555556,-5486.0,9.0
2,100013,1,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,10897.898516,1689174.270,6.165,357347.745,9740.235774,1509736.545,-14.0,-1358.109677,-210507.0,155.0
3,100028,0,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,4979.282257,562658.895,1.170,38988.540,4356.731549,492310.665,-29.0,-858.548673,-97016.0,113.0
4,100038,1,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,11100.337500,133204.050,11097.450,11100.600,11100.337500,133204.050,-466.0,-634.250000,-7611.0,12.0


In [30]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 666 entries, SK_ID_CURR to INSTAL_COUNT
dtypes: bool(144), float64(480), int64(42)
memory usage: 200.8 MB


## Credit_Card

In [31]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import gc

# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows=None, nan_as_category=True):
    cc = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\credit_card_balance.csv', nrows=num_rows)
    cc, cat_cols,cc_suppr = one_hot_encoder(cc, nan_as_category=True)
    
    # Drop SK_ID_PREV
    cc.drop(['SK_ID_PREV'], axis=1, inplace=True)

    # General aggregations
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])

    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()

    # Identify columns that should not be scaled (categorical + SK_ID_CURR)
    encoded_columns = cat_cols  # Assuming cat_cols contains all categorical columns
    non_categorical_cols = cc_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])  # Exclure SK_ID_CURR
    

    

    del cc
    gc.collect()

    return cc_agg


In [32]:
df_credit_card=credit_card_balance(num_rows = None, nan_as_category = True)

In [33]:
df_credit_card.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103558 entries, 100006 to 456250
Columns: 141 entries, CC_MONTHS_BALANCE_MIN to CC_COUNT
dtypes: bool(16), float64(101), int64(24)
memory usage: 101.1 MB


In [34]:
df_data = df_data.join(df_credit_card, how='left', on='SK_ID_CURR')

In [35]:
df_data.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100001,0,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100005,1,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100013,1,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,96.0
3,100028,0,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,49.0
4,100038,1,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 807 entries, SK_ID_CURR to CC_COUNT
dtypes: bool(144), float64(605), int64(42), object(16)
memory usage: 253.3+ MB


In [37]:
# Vérifier les valeurs infinies
infinity_mask = (df_data == np.inf) | (df_data == -np.inf)

# Afficher les colonnes contenant des valeurs infinies
infinite_columns = infinity_mask.any()

for col in df_data.columns[infinite_columns]:
    col_index = df_data.columns.get_loc(col)
    print(f"Colonne: {col}, Numéro dans le DataFrame initial: {col_index}")

# Créer df_data2 en supprimant les colonnes contenant des valeurs infinies
df_data2 = df_data.loc[:, ~infinite_columns]

In [38]:
df_data2.shape

(48744, 807)

In [39]:
# Reduction de l'échantillon pour gagner en vitesse de calcul pour train
# df_data2_sample = df_data2.sample(frac=0.10, random_state=42) # si je prend une partie
df_data2_sample=df_data2

In [40]:
df_data2_sample=df_data2

In [41]:
df_data2_sample.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100001,0,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100005,1,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100013,1,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,96.0
3,100028,0,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,49.0
4,100038,1,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_data2.shape

(48744, 807)

In [43]:
# Préparation sélection des donées X et Y

X = df_data2_sample.iloc[:, 2:]  # Toutes les colonnes sauf les deux premières
Y = df_data2_sample.iloc[:, 1]  # Colonne Tarket (variable cible)

In [44]:
# Répartition des données entraintement et test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=76, test_size=0.25)

In [45]:
X_train.head()

,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
46306,0,1,2,135000.0,157500.0,10656.0,157500.0,0.024610,-10349,-3608.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,17.0
22012,0,0,2,247500.0,213934.5,22594.5,193500.0,0.007020,-14757,-5751.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,36.0
11746,0,0,0,144000.0,247500.0,24610.5,247500.0,0.014520,-16891,-970.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35518,0,0,1,90000.0,431280.0,20745.0,360000.0,0.031329,-18186,-4381.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40763,1,0,0,211500.0,252000.0,23242.5,252000.0,0.018029,-9105,-1579.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
Y_train.head()

46306    0
22012    1
11746    0
35518    0
40763    1
Name: CODE_GENDER, dtype: int64

## réduction aux features sélectionnées

In [47]:
# Charger les caractéristiques sélectionnées depuis le fichier CSV

selected_features = pd.read_csv("selected_features.csv", header=None)

In [48]:
selected_features.head

<bound method NDFrame.head of              0           1              2            3                4   \
0  CNT_CHILDREN  DAYS_BIRTH  DAYS_EMPLOYED  OWN_CAR_AGE  FLAG_WORK_PHONE   

                5                            6             7             8   \
0  CNT_FAM_MEMBERS  REGION_RATING_CLIENT_W_CITY  EXT_SOURCE_1  EXT_SOURCE_2   

             9   ...                     15                              16  \
0  EXT_SOURCE_3  ...  PREV_CNT_PAYMENT_MEAN  PREV_NAME_CLIENT_TYPE_New_MEAN   

                             17                              18  \
0  APPROVED_APP_CREDIT_PERC_MIN  APPROVED_RATE_DOWN_PAYMENT_MIN   

                       19                                     20  \
0  POS_MONTHS_BALANCE_MAX  INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE   

               21                      22                      23  \
0  INSTAL_DPD_MAX  INSTAL_AMT_PAYMENT_MIN  INSTAL_AMT_PAYMENT_SUM   

                                24  
0  CC_AMT_DRAWINGS_ATM_CURRENT_SUM  

[1 rows x 25 

In [49]:
missing_columns = [col for col in selected_features if col not in X_train.columns]
if missing_columns:
    print(f"Colonnes manquantes : {missing_columns}")

Colonnes manquantes : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [50]:
type(X_train)

pandas.core.frame.DataFrame

In [51]:
print(selected_features)
print(type(selected_features))


             0           1              2            3                4   \
0  CNT_CHILDREN  DAYS_BIRTH  DAYS_EMPLOYED  OWN_CAR_AGE  FLAG_WORK_PHONE   

                5                            6             7             8   \
0  CNT_FAM_MEMBERS  REGION_RATING_CLIENT_W_CITY  EXT_SOURCE_1  EXT_SOURCE_2   

             9   ...                     15                              16  \
0  EXT_SOURCE_3  ...  PREV_CNT_PAYMENT_MEAN  PREV_NAME_CLIENT_TYPE_New_MEAN   

                             17                              18  \
0  APPROVED_APP_CREDIT_PERC_MIN  APPROVED_RATE_DOWN_PAYMENT_MIN   

                       19                                     20  \
0  POS_MONTHS_BALANCE_MAX  INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE   

               21                      22                      23  \
0  INSTAL_DPD_MAX  INSTAL_AMT_PAYMENT_MIN  INSTAL_AMT_PAYMENT_SUM   

                                24  
0  CC_AMT_DRAWINGS_ATM_CURRENT_SUM  

[1 rows x 25 columns]
<class 'pandas.core.f

In [52]:
# Extraire les noms des colonnes de selected_features
selected_features_list = selected_features.iloc[0].tolist()

# Vérifier si toutes les colonnes existent dans X_train
missing_columns = [col for col in selected_features_list if col not in X_train.columns]
if missing_columns:
    print(f"Colonnes manquantes dans X_train : {missing_columns}")
else:
    # Filtrer X_train avec les colonnes sélectionnées
    df_data2_sample_filtered = X_train[selected_features_list]
    print("Filtrage effectué avec succès.")


Filtrage effectué avec succès.


In [53]:
df_data2_sample_filtered.shape

(36558, 25)

In [54]:
# rajouter la colonne Target
df_data2_sample_filtered["TARGET"] = Y_train

In [55]:
df_data2_sample_filtered.head()

,CNT_CHILDREN,DAYS_BIRTH,DAYS_EMPLOYED,OWN_CAR_AGE,FLAG_WORK_PHONE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT_W_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,...,PREV_NAME_CLIENT_TYPE_New_MEAN,APPROVED_APP_CREDIT_PERC_MIN,APPROVED_RATE_DOWN_PAYMENT_MIN,POS_MONTHS_BALANCE_MAX,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DPD_MAX,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_SUM,CC_AMT_DRAWINGS_ATM_CURRENT_SUM,TARGET
46306,2,-10349,-3608.0,NaN,0,3.0,2,0.511762,0.667579,0.156640,...,0.200000,0.919143,0.000000,-10.0,2.0,7.0,143.910,510546.375,0.0,0
22012,2,-14757,-5751.0,NaN,0,4.0,2,0.482605,0.660309,0.450747,...,0.076923,0.000000,0.000000,-1.0,3.0,10.0,16.695,2924252.775,472500.0,1
11746,0,-16891,-970.0,NaN,0,2.0,2,NaN,0.622271,0.733815,...,0.333333,0.877684,0.093424,-2.0,1.0,44.0,1.350,586393.290,NaN,0
35518,1,-18186,-4381.0,NaN,0,3.0,2,NaN,0.563785,0.531686,...,0.000000,1.111111,0.108909,-51.0,1.0,0.0,6143.895,36871.875,NaN,0
40763,0,-9105,-1579.0,13.0,0,1.0,3,0.136241,0.268036,0.336062,...,0.100000,0.877463,0.000000,-11.0,3.0,5.0,3920.940,236219.940,NaN,1


In [56]:
df_data2_sample_filtered.shape

(36558, 26)

In [57]:
df_data2_sample_filtered.to_csv("df_data_test_non_scalées.csv", index=False)
print("Les données filtrées avec la colonne 'target' ont été sauvegardées dans 'df_data2_sample_filtered_with_target.csv'.")

Les données filtrées avec la colonne 'target' ont été sauvegardées dans 'df_data2_sample_filtered_with_target.csv'.
